# Prétraitements textuelles avec NLTK 

Import des différentes bibliothèques:

NLTK, unicodedata, contractions, inflect : Librairies orienté traitement de texte
Scikit-Learn : Librairies orientés machine learning

NLTK demande le téléchargement de certaints jeux de données pour pouvoir les utiliser, par exemple les stopwords.

In [1]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import unicodedata
import contractions
import inflect
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import *
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package stopwords to /home/alcoz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/alcoz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alcoz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/alcoz/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /home/alcoz/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

## Import des datasets organisés en dossier avec le label comme nom du dossier.
Fonction load_files (Scikit-Learn), l'encodage en UTF-8 est important, les fichiers sont importé en bytes.

La fonction utilise une structure Bunch, les attributs interessant sont :
   * data : Le tableau des différents texte avec une row est ègale à un fichier
   * target : Le tableau des labels encodé en entier numérique
   * target_names : Tableau des labels en String

In [2]:
Datasets_Train = load_files("../Datasets/Movies_Comment/train", encoding = 'UTF-8')
Datasets_Test = load_files("../Datasets/Movies_Comment/test", encoding = 'UTF-8')

In [3]:
#Commentaire
dataRaw = Datasets_Test.data
print(Datasets_Test.data[0])

#Label encodé
print(Datasets_Test.target[0])

#Label non encodé
print(Datasets_Test.target_names[Datasets_Test.target[0]])

My boyfriend and I went to watch The Guardian.At first I didn't want to watch it, but I loved the movie- It was definitely the best movie I have seen in sometime.They portrayed the USCG very well, it really showed me what they do and I think they should really be appreciated more.Not only did it teach but it was a really good movie. The movie shows what the really do and how hard the job is.I think being a USCG would be challenging and very scary. It was a great movie all around. I would suggest this movie for anyone to see.The ending broke my heart but I know why he did it. The storyline was great I give it 2 thumbs up. I cried it was very emotional, I would give it a 20 if I could!
1
pos


# Prétraitements avec nltk

Génération des StopWords et des GoWords avec extension des StopWords par des mots choisis par l'utilisateur.

In [4]:
GoWords = ['not', 'nor', 'up', 'out', 'can']
global OurStopWords
OurStopWords = ['movie', 'popcorn','the']

stop_words = stopwords.words('english')
stop_words.extend(OurStopWords)
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Nltk est une bibliothèque de fonction utilisable pour le traitement de texte.
### Pour pouvoir l'exploiter plus facilement, on va créer une fonction de pré-traitements et l'utiliser avec un vectorizer, un outils qui transforme les phrases en matrice de probabilité.

## Stemmatisation et Lemmatisation
La **Stemmatisation** (*ST*) et la **Lemmatisation** (*L*) sont deux fonctions qui ont pour objectif de réduire un mot pour en générer un version raccourci.

La *ST* va réduire un mot en supprimant les préfixes et les suffixes, la sortie représente la racine du mot.

La *L* consiste à ramener un terme à sa forme la plus simple (infinitif pour les verbes, Masculin/Singulier pour les noms, etc.)

La *ST* ne vas pas forcément renvoyé un mot existant alors que la *L* oui

Il existe plusieurs moteurs pour la *ST*, Ici on utilise le Snowball.
Sa particularité est qu'il est crée pour plusieurs langues.

## Part Of Speech (*POS*)

Liste des catégories de mots :
* CC coordinating conjunction
* CD cardinal digit
* DT determiner
* EX existential there (like: “there is” … think of it like “there exists”)
* FW foreign word
* IN preposition/subordinating conjunction
* JJ adjective ‘big’
* JJR adjective, comparative ‘bigger’
* JJS adjective, superlative ‘biggest’
* LS list marker 1)
* MD modal could, will
* NN noun, singular ‘desk’
* NNS noun plural ‘desks’
* NNP proper noun, singular ‘Harrison’
* NNPS proper noun, plural ‘Americans’
* PDT predeterminer ‘all the kids’
* POS possessive ending parent’s
* PRP personal pronoun I, he, she
* PRP\$ possessive pronoun my, his, hers
* RB adverb very, silently,
* RBR adverb, comparative better
* RBS adverb, superlative best
* RP particle give up
* TO, to go ‘to’ the store.
* UH interjection, errrrrrrrm
* VB verb, base form take
* VBD verb, past tense took
* VBG verb, gerund/present participle taking
* VBN verb, past participle taken
* VBP verb, sing. present, non-3d take
* VBZ verb, 3rd person sing. present takes
* WDT wh-determiner which
* WP wh-pronoun who, what
* WP\$ possessive wh-pronoun whose
* WRB wh-abverb where, when

In [5]:
def clean_text(commentString):   
    # Removing non ASCII characters
    commentString = unicodedata.normalize('NFKD', commentString).encode("ascii", "ignore").decode("utf-8", 'ignore')

    # Removing contractions
    commentString = contractions.fix(commentString, slang = True)

    # Tokenizing
    tokenizedText = word_tokenize(commentString)

    # Putting all words in lowercase
    tokenizedText = [word.lower() for word in tokenizedText]

    #Putting all words in uppercase
    #tokenizedText = [word.upper() for word in tokenizedText]
    
    # Deleting ponctuations
    tokenizedText = [word for word in tokenizedText if word.isalpha()]

    # Removing stop words
    tokenizedText = [word for word in tokenizedText if not word in stop_words]
    
    # Converting numbers
    inflectEngine = inflect.engine()
    newWords = []
    for word in tokenizedText:
        if word.isdigit():
            newWords.append(inflectEngine.number_to_words(word))
        else:
            newWords.append(word)
    tokenizedText = newWords

    # Stemmatisation
    stemmer = EnglishStemmer()
    commentString = [stemmer.stem(word) for word in tokenizedText]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    commentString = [lemmatizer.lemmatize(word) for word in tokenizedText]

    # Turning back tokens into a string
    commentString = "".join([" " + i for i in tokenizedText]).strip()
    
    return commentString

In [6]:
print(SnowballStemmer.languages)

('arabic', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'hungarian', 'italian', 'norwegian', 'porter', 'portuguese', 'romanian', 'russian', 'spanish', 'swedish')


In [7]:
print(dataRaw[0])
clean = clean_text(dataRaw[0])
print("================")
print(clean)
print(nltk.pos_tag(nltk.word_tokenize(clean)))

My boyfriend and I went to watch The Guardian.At first I didn't want to watch it, but I loved the movie- It was definitely the best movie I have seen in sometime.They portrayed the USCG very well, it really showed me what they do and I think they should really be appreciated more.Not only did it teach but it was a really good movie. The movie shows what the really do and how hard the job is.I think being a USCG would be challenging and very scary. It was a great movie all around. I would suggest this movie for anyone to see.The ending broke my heart but I know why he did it. The storyline was great I give it 2 thumbs up. I cried it was very emotional, I would give it a 20 if I could!
boyfriend went watch first want watch loved definitely best seen portrayed uscg well really showed think really appreciated teach really good shows really hard job think uscg would challenging scary great around would suggest anyone ending broke heart know storyline great give thumbs cried emotional would 

### NLTK Tag Helper : donne une explication pour chaque tag et des exemples

In [8]:
nltk.help.upenn_tagset('RB')

RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...


## Application d'un vectorizer sur nos textes

Les vectorizer permettent de calculer des occurences ou des calculs sur les occurences de mots ou de groupes de mots sur une liste de documents.
Avec Scikit-Learn, on peut utiliser une fonction de prétraitement sur nos données avant d'appliquer la vectorisation.
Les paramètres les plus importants sont:
   * les N-grams: valeurs représentant le nombre de mots étudiés, si c'est un ensemble. La fonction prend un mots, ensuits le suivant, et ainsi de suite jusqu'au nombre choisis comme deuxième valeur.
   * min_df et max-df: le vectoriseur va sélectionner que les valeurs appartenat à cette ensemble.<br>

Les 3 vectorizer présents sur Scikit-Learn:
   * CountVectorizer : génère une matrice d'occurences des mots pour une liste de documents
   * HashingVectorizer : Comme le count mais en utilisant une méthode de Hachage pour créer le tableau
   * TfidfVectorizer : gènère une matrice TF-IDF pour la collection documents

In [9]:
vectorizer = TfidfVectorizer(preprocessor = clean_text, ngram_range = (1, 3), min_df = 0.01, max_df = 0.9)
vectors = vectorizer.fit_transform(dataRaw)

data = vectors.toarray()

# Différentes bibliothèques utiles pour le NLP

**Pattern :** Une bibliothèque regroupant des outils de DataMining, de NLP, de Machine Learning et des modèles de graphe. <br>
GitHUB : https://github.com/clips/pattern/wiki

**Spacy** : Une librarie au top de la recherche dans le NLT et orienté à l'implémentation de cas concret. La bibliothèque est composé de modèle statistiques pré-entrainé, des vecteurs de mots et des fonctions de tokenisation dans plus de 60 langues. <br>
La bibliothèque est exploitable pour des conv networks, le parsage reconnaissance d'entité et des méthdoe de Deep Learning <br>
GitHUB : https://github.com/explosion/spaCy